# Models

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
import pandas as pd

In [2]:
from pyspark.ml import Pipeline  
from pyspark.ml.feature import *  
from pyspark.ml.classification import LogisticRegression

In [3]:
df = spark.read.csv('train_data.csv', inferSchema=True, header=True)

In [4]:
dfs = df.select([col for col in df.columns if col not in ['addr_state', 'term','_c0', 'emp_length_low']])

In [5]:
dfs.show(5)

+------------------+---------+--------+--------------+-----------+-----------+----------+--------------+---------------+--------+---------+
|loan_status_binary|loan_amnt|int_rate|home_ownership|tot_cur_bal|total_pymnt|annual_inc|fico_range_low|last_pymnt_amnt|grade_CD|grade_EFG|
+------------------+---------+--------+--------------+-----------+-----------+----------+--------------+---------------+--------+---------+
|                 0|  11000.0|    7.21|           ANY|    28511.0|    1354.03|   40000.0|         715.0|         340.71|       0|        0|
|                 0|   4000.0|    22.9|           ANY|   108997.0|    1386.67|   40000.0|         665.0|         154.64|       0|        1|
|                 0|  10000.0|   17.97|           ANY|    20320.0|    1435.54|   60000.0|         665.0|         361.38|       1|        0|
|                 0|  13000.0|   10.91|           ANY|    34947.0|  14686.205|   45000.0|         725.0|        7470.61|       0|        0|
|                 0|

In [6]:
train, test = dfs.randomSplit([0.8, 0.2], seed=12345)

In [7]:
stringIndexer = StringIndexer(inputCol="home_ownership", outputCol="homeIndex")

In [8]:
encoder = OneHotEncoder(inputCol="homeIndex", outputCol="homeVec")

In [9]:
scaled_vectors = VectorAssembler(inputCols = ['loan_amnt', 'tot_cur_bal',
                                             'total_pymnt', 'annual_inc', 
                                             'fico_range_low', 'last_pymnt_amnt',
                                             'int_rate'], outputCol='vector_features')

In [10]:
output = scaled_vectors.transform(train)

In [11]:
scaler = StandardScaler(inputCol='vector_features', outputCol='scaled_features')

In [12]:
assembler = VectorAssembler(inputCols=['scaled_features', 'homeVec', 'grade_CD', 
                                       'grade_EFG'], outputCol='features')

In [13]:
lr = LogisticRegression(maxIter=10, regParam=0.01)

In [14]:
pipeline = Pipeline(stages=[stringIndexer, encoder, scaler,
                            assembler, lr])

In [15]:
# Fit the pipeline
model = pipeline.fit(output)

# Make a prediction
prediction = model.transform(test)

Py4JJavaError: An error occurred while calling o66.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 5.0 failed 1 times, most recent failure: Lost task 1.0 in stage 5.0 (TID 9, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<loan_amnt:double,tot_cur_bal:double,total_pymnt:double,annual_inc:double,fico_range_low:double,last_pymnt_amnt:double,int_rate:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iter